# Setup

Provides a function which is used by all remaining cells.

In [ ]:
import datetime
import hpq


def format(obj):
    dt = datetime.datetime.utcfromtimestamp(obj["last_tradable_date"] / 1000000000)
    obj["last_tradable_date"] = dt.strftime("%Y-%m-%d")
    #   Always false for metadata
    del obj["implied"]
    return hpq.format(obj)

# Filter

Downloads all metadata for all products for the trading date of 2022-09-16 for the CME selecting only those which are spreads with exactly the following product IDs as legs:

- 293009
- 457947
- 514348

Then displays the matching metadata in a table. Note that downloading and filtering all metadata for all products on the CME may take some time as there are hundreds of thousands of products on the CME each day.

In [ ]:
import hpq
import ijson
import IPython.display
import tabulate

connection = hpq.create_web_socket_client()
desired = [293009, 457947, 514348]


def predicate(product):
    if not "legs" in product.keys():
        return False
    legs = product["legs"]
    if len(legs) != 3:
        return False
    for leg in legs:
        if leg["product_id"] not in desired:
            return False
    return True


def transform(product):
    #   Doesn't display well in table since it's an array
    del product["legs"]
    return format(product)


products = list(
    map(
        transform,
        filter(
            predicate,
            ijson.items(
                connection.stream(
                    {
                        "query": "products",
                        "source": ["cme"],
                        "date": "2022-09-16",
                        "type": "metadata",
                    }
                ),
                "item",
            ),
        ),
    )
)
table = tabulate.tabulate(products, tablefmt="html", headers="keys")
IPython.display.display(IPython.display.HTML(table))

# Snapshot

Retrieves a summary of the metadata for products with the following IDs:

- 293009
- 457947
- 514348

as of 2022-09-16T17:30:00ET. Note:

- The above product IDs are the same product IDs which were expected as legs in the preceding cell
- Specifying not just a date but a time is necessary when retrieving product metadata due to the fact metadata may be updated intra-day

Once retrieved the product metadata will be formatted as a table. Note that the cell makes three separate requests to the HPQ API but the same connection is reused for all three.

In [ ]:
import hpq
import ijson
import IPython.display
import tabulate

connection = hpq.create_web_socket_client()
products = []
for id in [293009, 457947, 514348]:
    for metadata in ijson.items(
        connection.stream(
            {
                "query": "snapshot",
                "type": "productinfo",
                "date": "2022-09-16",
                "time": "17:30:00",
                "source": "cme",
                "product": str(id),
            }
        ),
        "item",
    ):
        #   Doesn't display well in table since it's an array
        del metadata["underlyings"]
        products.append(format(metadata))
table = tabulate.tabulate(products, tablefmt="html", headers="keys")
IPython.display.display(IPython.display.HTML(table))